In [ ]:
from torchvision import datasets, transforms
import numpy as np
import random
from sklearn.preprocessing import LabelBinarizer

import tensorflow as tf
from keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import cv2
import os

from tqdm import tqdm

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [ ]:
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5,)),
])
transform_train_minor = transforms.Compose([
    transforms.RandomPerspective(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),])

In [ ]:
dataset_train = datasets.FashionMNIST('../data/fashion-mnist', train=True, download = True, transform = transform_train)
dataset_test = datasets.FashionMNIST('../data/fashion-mnist-t',train=False, download = True, transform = transform_train)

In [ ]:
#this function is sampling the cifar image index in non iid way, each client will have majority of 2 class and 8 minor class
def fashion_mnist_noniid2(dataset, num_users, p):

    idxs = np.arange(len(dataset),dtype=int)
    labels = np.array(dataset.targets)
    label_list = np.unique(dataset.targets)
    
    # sort labels
    idxs_labels = np.vstack((idxs, labels))
    idxs_labels = idxs_labels[:,idxs_labels[1,:].argsort()]
    #print(idxs_labels)
    idxs = idxs_labels[0,:]
    idxs = idxs.astype(int)
    n_data=int(len(dataset)/(num_users+1))
    dict_users = {i: np.array([], dtype='int64') for i in range(num_users)}

    #Sample majority class for each user
    user_majority_labels = []
    for i in range(num_users):
        majority_labels = np.random.choice(label_list, 2, replace = False)
        user_majority_labels.append(majority_labels)
       # print(i,majority_labels)
        majority_label_idxs = (majority_labels[0] == labels[idxs]) | (majority_labels[1] == labels[idxs])        
        
        sub_data_idxs = np.random.choice(idxs[majority_label_idxs], int(p*n_data), replace = False)
        
        dict_users[i] = np.concatenate((dict_users[i],sub_data_idxs))
        idxs = np.array(list(set(idxs) - set(sub_data_idxs)))
        
        #assigning minor classes to each client
    if(p < 1.0):
        for i in range(num_users):
            majority_labels = user_majority_labels[i]
            
            non_majority_label_idxs = (majority_labels[0] != labels[idxs]) & (majority_labels[1] != labels[idxs])
            
            sub_data_idxs = np.random.choice(idxs[non_majority_label_idxs], int((1-p)*n_data), replace = False)
            
            dict_users[i] = np.concatenate((dict_users[i], sub_data_idxs))
            idxs = np.array(list(set(idxs) - set(sub_data_idxs)))
    idx=int((1-p)*n_data)
    return dict_users,idx

In [ ]:
num_user=10
p=0.4
dict_users,idx = fashion_mnist_noniid2(dataset_train,num_user,p)
print('done sampling')

In [ ]:
#using image augmentation twice and add 2 transformed image
def augment_image(dataset,idx,num_user):
  dataset=list(dataset)
  count=len(dataset)
  final_idx=len(dataset)
  for j in tqdm(range(num_user)):
      for i in dict_users[j][idx:final_idx]: #starting from 400 because after 400 array will conatin indexes of minor class image 
          label=dataset[i][1]
          new_image1=cv2.blur(dataset[i][0].numpy(),(2,2))
          dataset.append([new_image1,label])   
          count += 1
          dict_users[j] = np.append(dict_users[j], count)
          new_image2=transform_train_minor(dataset[i][0])
          dataset.append([new_image2,label])   
          count += 1
          dict_users[j] = np.append(dict_users[j], count)  
  return dict_users,dataset

In [ ]:
dict_users,dataset_train=augment_image(dataset_train,idx,num_user)

In [ ]:
# code to check number of images/classes
temp=list()
for i in dict_users[0]:
     temp.append(dataset_train[i][1])

for i in range(10):
     print("class",i,'->',temp.count(i))

In [ ]:
5453

In [ ]:
#creating a batch data of 2500 imgs per client
lb=LabelBinarizer()
def batch_data(data_shard, bs=5453):
    label=[]
    for i in range(len(data_shard)):
        label.append(dataset_train[i][1])
    label=lb.fit_transform(label)
    data=[]
    for i in range(len(data_shard)):
        data.append(dataset_train1[i]) 
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(bs)
print('done label binarizing')

In [ ]:
ten_datatype=type(dataset_train[0][0])          
#changing the tensor datatype to numpy datatype
dataset_train1=[]
for i in range(len(dataset_train)):
    if(type(dataset_train[i][0])==ten_datatype):
        temp=dataset_train[i][0].numpy()
        dataset_train1.append(np.transpose(temp))

#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in dict_users.items():
    clients_batched[client_name] = batch_data(data)


In [ ]:
testset1=[]
for i in range(5000):
    temp=dataset_test[i][0].numpy()
    testset1.append(np.transpose(temp))  
#process and batch the test set  
label=[]
data=[]
for i in tqdm(range(5000)): #getting 2000 images only from testset to test the model
    label.append(dataset_test[i][1])
    data.append(testset1[i]) 
label=lb.fit_transform(label)
      
test_batched = tf.data.Dataset.from_tensor_slices((list(data), list(label))).batch(5000)

del dataset_train
del dataset_train1

In [ ]:
#building a simple NN
class SimpleMLP:
    @staticmethod
    def build(classes):
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(28,28,1)))
        model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(0.3))
        model.add(Flatten())
        model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dropout(0.3))
        model.add(Dense(classes, activation='softmax'))
        model.compile(loss='categorical_crossentropy',
                      optimizer=SGD(lr=0.001, decay=0.0002, momentum=0.9), 
                      metrics=['accuracy'])
        return model

In [ ]:
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad


def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    #logits = model.predict(X_test, batch_size=100)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [ ]:
#initialize global model

smlp_global = SimpleMLP()
global_model = smlp_global.build(10)
comm_round=30      
#commence global training loop
for comm_round in range(comm_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    #print(global_weights)
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    
    #loop through each client and create new local model
    for client in client_names:
        with strategy.scope():
          smlp_local = SimpleMLP()
          local_model = smlp_local.build(10)
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=10, verbose=0,validation_data=test_batched)
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)